In [3]:
import pandas as pd
import numpy as np  
import seaborn as sns
import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string
from wordcloud import STOPWORDS

In [ ]:

df = pd.read_pickle("df.pkl")

# Traitement des valeurs nuls ou non pertinentes

In [5]:
df.drop_duplicates(inplace=True)


In [6]:
print(df.shape)

(5080, 15)


In [7]:
df.dropna(inplace=True)
print(df.shape) 
df.info()

(5080, 15)
<class 'pandas.core.frame.DataFrame'>
Index: 5080 entries, 31 to 7581
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5080 non-null   int64  
 1   keyword            5080 non-null   object 
 2   location           5080 non-null   object 
 3   text               5080 non-null   object 
 4   target             5080 non-null   int64  
 5   char_count         5080 non-null   int64  
 6   word_count         5080 non-null   int64  
 7   stopword_count     5080 non-null   int64  
 8   unique_word_count  5080 non-null   int64  
 9   stop_word_count    5080 non-null   int64  
 10  url_count          5080 non-null   int64  
 11  mean_word_length   5080 non-null   float64
 12  punctuation_count  5080 non-null   int64  
 13  hashtag_count      5080 non-null   int64  
 14  mention_count      5080 non-null   int64  
dtypes: float64(1), int64(11), object(3)
memory usage: 635.0+ KB


In [8]:
df.head()

,id,keyword,location,text,target,char_count,word_count,stopword_count,unique_word_count,stop_word_count,url_count,mean_word_length,punctuation_count,hashtag_count,mention_count
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,55,5,0,5,0,1,10.200000,6,0,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,67,10,4,10,3,1,5.800000,8,2,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,82,9,1,9,1,1,8.222222,9,1,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,34,7,3,7,3,0,4.000000,1,0,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,76,13,7,13,5,1,4.923077,5,0,0


In [9]:
df['target'].value_counts(normalize=True)

target
0    0.567717
1    0.432283
Name: proportion, dtype: float64

# Proportion target

In [10]:

df_0 = df[df['target'] == 0]
df_1 = df[df['target'] == 1]

# 2. Échantillonner aléatoirement la classe 0 pour avoir autant d'exemples que la classe 1
df_0_sampled = df_0.sample(n=len(df_1), random_state=42)

df_balanced = pd.concat([df_0_sampled, df_1], axis=0).sample(frac=1, random_state=42)  # shuffle le tout


In [11]:

df_balanced['target'].value_counts(normalize=True)

target
1    0.5
0    0.5
Name: proportion, dtype: float64

In [12]:
df_balanced['keyword'].value_counts()


keyword
collision                33
outbreak                 31
sandstorm                31
fatalities               30
emergency%20plan         29
                         ..
razed                     9
detonation                8
epicentre                 7
radiation%20emergency     6
inundation                3
Name: count, Length: 221, dtype: int64

## Nettoyage

Passage en miniscule

In [13]:
df_balanced['text'] = df_balanced['text'].str.lower()
df_balanced['keyword'] = df_balanced['keyword'].str.lower()
df_balanced['location'] = df_balanced['location'].str.lower()

In [14]:
df_balanced.head()

,id,keyword,location,text,target,char_count,word_count,stopword_count,unique_word_count,stop_word_count,url_count,mean_word_length,punctuation_count,hashtag_count,mention_count
6818,9765,trapped,10 steps ahead. cloud 9,bomb head? explosive decisions dat produced mo...,1,134,21,6,20,6,0,5.428571,1,0,0
4842,6896,mass%20murder,"huntsville, al",okay not sure the word 'mass murder' applies d...,1,121,19,9,18,8,1,5.421053,8,0,0
5848,8356,ruin,garrett,like why on earth would you want anybody to be...,0,101,17,6,17,8,0,5.000000,2,0,0
1851,2661,crush,"cleveland, ohio",my woman crush wedneday goes to the beautiful ...,0,108,12,3,12,3,1,8.083333,8,2,1
5729,8176,rescuers,washington,#news: 'many deaths' in shipwreck: rescuers ar...,1,136,18,6,18,5,1,6.611111,14,1,1


Suppression liens /mentions / hashtags / ponctuation/chiffres

Suppression des stopwords et vectorisation


In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)      # URLs
    text = re.sub(r"@\w+|#\w+", "", text)           # Mentions & hashtags
    text = re.sub(r"[^a-z\s]", "", text)            # Ponctuation / chiffres
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Nettoyage de base
df_balanced["clean_text"] = df_balanced["text"].apply(clean_text)

# Tokenisation X
df_balanced["tokens"] = df_balanced["clean_text"].apply(lambda x: x.split())

# Suppression des stopwords
df_balanced["tokens"] = df_balanced["tokens"].apply(lambda tokens: [w for w in tokens if w not in stop_words])  


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sulivanmoreau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Application 

In [16]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()


# Version 2 : Stopwords uniquement
df_balanced["version_stop"] = df_balanced["tokens"].apply(lambda x: " ".join(x))

# Version 3 : Stemming
df_balanced["version_stem"] = df_balanced["tokens"].apply(
    lambda tokens: " ".join([stemmer.stem(t) for t in tokens])
)

# Version 4 : Lemmatization
df_balanced["version_lemma"] = df_balanced["tokens"].apply(
    lambda tokens: " ".join([lemmatizer.lemmatize(t) for t in tokens])
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sulivanmoreau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sulivanmoreau/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sulivanmoreau/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
df_balanced.head()

,id,keyword,location,text,target,char_count,word_count,stopword_count,unique_word_count,stop_word_count,url_count,mean_word_length,punctuation_count,hashtag_count,mention_count,clean_text,tokens,version_stop,version_stem,version_lemma
6818,9765,trapped,10 steps ahead. cloud 9,bomb head? explosive decisions dat produced mo...,1,134,21,6,20,6,0,5.428571,1,0,0,bomb head explosive decisions dat produced mor...,"[bomb, head, explosive, decisions, dat, produc...",bomb head explosive decisions dat produced dea...,bomb head explos decis dat produc dead childre...,bomb head explosive decision dat produced dead...
4842,6896,mass%20murder,"huntsville, al",okay not sure the word 'mass murder' applies d...,1,121,19,9,18,8,1,5.421053,8,0,0,okay not sure the word mass murder applies dur...,"[okay, sure, word, mass, murder, applies, war,...",okay sure word mass murder applies war horrend...,okay sure word mass murder appli war horrend n...,okay sure word mass murder applies war horrend...
5848,8356,ruin,garrett,like why on earth would you want anybody to be...,0,101,17,6,17,8,0,5.000000,2,0,0,like why on earth would you want anybody to be...,"[like, earth, would, want, anybody, unhappy, d...",like earth would want anybody unhappy dont pur...,like earth would want anybodi unhappi dont pur...,like earth would want anybody unhappy dont pur...
1851,2661,crush,"cleveland, ohio",my woman crush wedneday goes to the beautiful ...,0,108,12,3,12,3,1,8.083333,8,2,1,my woman crush wedneday goes to the beautiful,"[woman, crush, wedneday, goes, beautiful]",woman crush wedneday goes beautiful,woman crush wedneday goe beauti,woman crush wedneday go beautiful
5729,8176,rescuers,washington,#news: 'many deaths' in shipwreck: rescuers ar...,1,136,18,6,18,5,1,6.611111,14,1,1,many deaths in shipwreck rescuers are trying t...,"[many, deaths, shipwreck, rescuers, trying, sa...",many deaths shipwreck rescuers trying save hun...,mani death shipwreck rescuer tri save hundr mi...,many death shipwreck rescuer trying save hundr...


In [19]:
df_balanced.to_pickle("df_balanced.pkl")